# CSCI 4253 / 5253 - Lab #4 - Patent Problem with Spark DataFrames
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

This [Spark cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf) is useful as is [this reference on doing joins in Spark dataframe](http://www.learnbymarketing.com/1100/pyspark-joins-by-example/).

The [DataBricks company has one of the better reference manuals for PySpark](https://docs.databricks.com/spark/latest/dataframes-datasets/index.html) -- they show you how to perform numerous common data operations such as joins, aggregation operations following `groupBy` and the like.

In [13]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

The following aggregation functions may be useful -- [these can be used to aggregate results of `groupby` operations](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#example-aggregations-using-agg-and-countdistinct). More documentation is at the [PySpark SQL Functions manual](https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html#module-pyspark.sql.functions). Feel free to use other functions from that library.

In [14]:
from pyspark.sql.functions import col, count, countDistinct

Create our session as described in the tutorials

In [15]:
spark = SparkSession \
    .builder \
    .appName("Lab4-Dataframe") \
    .master("local[*]")\
    .getOrCreate()

Read in the citations and patents data and check that the data makes sense. Note that unlike in the RDD solution, the data is automatically inferred to be Integer() types.

In [16]:
citations = spark.read.load('cite75_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [17]:
citations.show(5)

+-------+-------+
| CITING|  CITED|
+-------+-------+
|3858241| 956203|
|3858241|1324234|
|3858241|3398406|
|3858241|3557384|
|3858241|3634889|
+-------+-------+
only showing top 5 rows



In [18]:
patents = spark.read.load('apat63_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [19]:
patents.show(5)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|3070801| 1963| 1096|   NULL|     BE|   NULL|    NULL|      1|  NULL|   269|  6|    69| NULL|       1|    NULL|    0.0|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|
|3070802| 1963| 1096|   NULL|     US|     TX|    NULL|      1|  NULL|     2|  6|    63| NULL|       0|    NULL|   NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|
|3070803| 1963| 1096|   NULL|     US|     IL|    NULL|      1|  NULL|     2|  6|    6

In [20]:
# === Lab 4 — DataFrame solution (Top-10 SAME_STATE like instructor PNG) ===
from pyspark.sql import SparkSession, functions as F
import glob

spark = SparkSession.builder.appName("lab4-df").getOrCreate()

# --- locate files in current dir OR ./data (works with your Make.txt) ---
def pick_one(pats):
    for p in pats:
        m = sorted(glob.glob(p))
        if m: return m[0]
    raise FileNotFoundError(pats)

patents_path   = pick_one(["./apat63_99.txt*", "./data/apat63_99.txt*"])
citations_path = pick_one(["./cite75_99.txt*", "./data/cite75_99.txt*", "./data/acite75_99.txt*"])
print("USING:", patents_path, "|", citations_path)






USING: ./apat63_99.txt.gz | ./cite75_99.txt.gz


In [21]:
# --- read patents (CSV with header) ---
patents_raw = (
    spark.read
         .option("header", "true")
         .option("sep", ",")
         .option("quote", '"').option("escape", '"')
         .option("mode", "DROPMALFORMED")
         .csv(patents_path)
).cache()

# --- read citations (try comma CSV; fall back to whitespace split) ---
try_csv = spark.read.option("header","false").option("sep",",").csv(citations_path)
if len(try_csv.columns) == 2:
    citations_raw = try_csv.toDF("CITING","CITED")
else:
    txt = spark.read.text(citations_path)
    citations_raw = (txt
        .select(F.split(F.col("value"), r"\s+").alias("c"))
        .select(F.col("c").getItem(0).alias("CITING"),
                F.col("c").getItem(1).alias("CITED"))
    )


In [22]:

# --- minimal normalized tables ---
# US patents with a real state → map: pid -> state
pmap = (patents_raw
    .select(F.col("PATENT").cast("string").alias("pid"),
            F.upper(F.trim("COUNTRY")).alias("country"),
            F.upper(F.trim("POSTATE")).alias("state"))
    .filter((F.col("country")=="US") & F.col("state").isNotNull() & (F.col("state")!=""))
).cache()

# citations as strings (distinct to be safe)
edges = (citations_raw
    .select(F.col("CITING").cast("string").alias("citing"),
            F.col("CITED").cast("string").alias("cited"))
    .distinct()
).cache()

# --- add states to both sides (inner joins so both states exist) ---
lk_citing = F.broadcast(pmap.select(F.col("pid").alias("citing_id"),
                                    F.col("state").alias("citing_state")))
lk_cited  = F.broadcast(pmap.select(F.col("pid").alias("cited_id"),
                                    F.col("state").alias("cited_state")))

pairs = (edges
    .join(lk_citing, F.col("citing")==F.col("citing_id"), "inner")
    .join(lk_cited,  F.col("cited")==F.col("cited_id"),  "inner")
    .select("citing","cited","citing_state","cited_state")
).cache()



In [23]:
# (nice for interview) show the 4-column intermediate
pairs.show(10, truncate=False)

# --- SAME_STATE per citing patent: sum of boolean (0/1) ---
same_by_citing = (pairs
    .withColumn("is_same", (F.col("citing_state")==F.col("cited_state")).cast("int"))
    .groupBy("citing").agg(F.sum("is_same").alias("SAME_STATE"))
)



+-------+-------+------------+-----------+
|citing |cited  |citing_state|cited_state|
+-------+-------+------------+-----------+
|3858260|3090973|IL          |TX         |
|3858359|3691689|IN          |IN         |
|3858623|3159530|NY          |WI         |
|3858836|3361515|CA          |NY         |
|3858950|3510182|OH          |CT         |
|3858950|3748003|OH          |IN         |
|3858955|3753023|NJ          |NJ         |
|3859122|3464922|NC          |NJ         |
|3859210|3575852|TX          |IL         |
|3859249|3316330|NC          |IL         |
+-------+-------+------------+-----------+
only showing top 10 rows



In [24]:
# --- augment the full patent rows with SAME_STATE (left join; fill 0) ---
patents_aug = (patents_raw
    .withColumn("PATENT_s", F.col("PATENT").cast("string"))
    .join(same_by_citing, F.col("PATENT_s")==F.col("citing"), "left")
    .drop("citing","PATENT_s")
    .withColumn("SAME_STATE", F.coalesce(F.col("SAME_STATE"), F.lit(0)))
).cache()

# --- print Top-10 like the instructor PNG (all original cols + SAME_STATE) ---
pat_cols = patents_raw.columns  # preserve original column order
(patents_aug
    .select(*[F.col(c) for c in pat_cols], F.col("SAME_STATE"))
    .orderBy(F.desc("SAME_STATE"), F.asc("PATENT"))
    .show(10, truncate=False))

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+----------+
|PATENT |GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|SAME_STATE|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+----------+
|5959466|1999 |14515|1997   |US     |CA     |5310    |2      |NULL  |326   |4  |46    |159  |0       |1       |NULL   |0.6186  |NULL    |4.8868  |0.0455  |0.044   |NULL    |NULL    |125       |
|5983822|1999 |14564|1998   |US     |TX     |569900  |2      |NULL  |114   |5  |55    |200  |0       |0.995   |NULL   |0.7201  |NULL    |12.45   |0       |0       |NULL    |NULL    |103       |
|6008204|1999 |14606|1998   |U